## Reconstruction image of GEM and HOPS particles

In [4]:
from tifffile import imwrite
from tkinter import filedialog as fd
import os
from os.path import dirname, basename
from scipy.ndimage import gaussian_filter
import numpy as np
import pandas as pd
from rich.progress import track
import matplotlib.pyplot as plt
import matplotlib

matplotlib.use("Agg")
pd.options.mode.chained_assignment = None  # default='warn'

In [5]:
print("Choose one or more '_reformatted.csv' files from single-channel SPT-PAINT experiment:")
lst_path = list(fd.askopenfilenames(filetypes=[("CSV files", "*_reformatted.csv")]))

if not lst_path:
    print("No file selected!")
else:
    folder_save = dirname(lst_path[0])
    os.chdir(folder_save)
    lst_fname = [basename(f) for f in lst_path if f.endswith("_reformatted.csv")]

    # Parameters for scaling and image dimensions
    um_per_pixel = 0.117
    scaling_factor = 1
    um_per_pixel_PAINT = um_per_pixel / scaling_factor
    xpixels_ONI = 428
    ypixels_ONI = 684
    xedges = np.arange((xpixels_ONI + 1)*scaling_factor)
    yedges = np.arange((ypixels_ONI + 1)*scaling_factor)

    def spots2PAINT(df):
        # This function reconstructs PAINT from the whole dataframe, assuming it covers the full FOV
        # Single-frame spots
        df_single_frame_spots = df[df["trackID"].isna()]
        img_spots, _, _ = np.histogram2d(
            x=df_single_frame_spots["x"].to_numpy(float) * scaling_factor,
            y=df_single_frame_spots["y"].to_numpy(float) * scaling_factor,
            bins=(xedges, yedges),
        )

        # Tracks
        df_tracks = df[df["trackID"].notna()]
        all_trackID = df_tracks["trackID"].unique()
        lst_of_arr_x = []
        lst_of_arr_y = []
        for trackID in track(all_trackID, description="Reconstruction: PAINT"):
            df_current = df_tracks[df_tracks["trackID"] == trackID]
            lst_of_arr_x.append(df_current["x"].to_numpy(float) * scaling_factor)
            lst_of_arr_y.append(df_current["y"].to_numpy(float) * scaling_factor)

        img_tracks, _, _ = np.histogram2d(
            x=np.hstack(lst_of_arr_x),
            y=np.hstack(lst_of_arr_y),
            bins=(xedges, yedges),
        )

        img_PAINT = img_spots + img_tracks

        return img_PAINT

    for fname_singlechannel in lst_fname:
        print("Now processing:", fname_singlechannel.split("-reformatted")[0])
        # Read data
        df_singlechannel = pd.read_csv(fname_singlechannel)
        
        # Reconstruct PAINT image
        img_PAINT_singlechannel = spots2PAINT(df_singlechannel)
        imwrite(fname_singlechannel.split("-reformatted")[0] + "-PAINT.tif", img_PAINT_singlechannel)

        img_denoise = gaussian_filter(img_PAINT_singlechannel, sigma=1)
        imwrite(fname_singlechannel.split("-reformatted")[0] + "-PAINT-smoothed.tif", img_denoise)

        # plt.figure(figsize=(6, 4))
        # plt.imshow(img_denoise, cmap='hot', interpolation='nearest')
        # plt.colorbar()
        # plt.show()

Choose one or more '_reformatted.csv' files from single-channel SPT-PAINT experiment:
Now processing: 20240613_UGD_ER_405_7_532_5_2x_rep1_spots_reformatted.csv


Output()

Now processing: 20240627_UGD_ER_405_7_532_5_2x_rep3_spots_reformatted.csv


Output()

Now processing: 20240716_UGD_ER_405_7_532_5_rep1_spots_reformatted.csv


Output()

Now processing: 20240716_UGD_ER_405_7_532_5_rep2_spots_reformatted.csv


Output()